### word2vecの高速化
* Embeddingレイヤの導入
* Negative Samplingという新しい損失関数の導入。

##### 問題点
* 入力層がone-hot表現のため語彙数が増えるとそれ相応のメモリサイズが必要になる。→Embedding層の追加
* 中間層の問題（多くの計算が必要になる）→Negative sampling 

#### Embeddingレイヤ
* 単語IDに該当する行を抜き出すレイヤ(**Embedding層**)
* 単語の密なベクトル表現を単語の埋め込み（Embedding）や単語の分散表現と呼ばれる。

#### 中間層の問題
* 多値分類から二値分類に変換する。（これはAであるか否かみたいな感じ）

##### シグモイド関数と交差エントロピー誤差
* 多値分類の場合は出力層にはソフトマックス関数、損失関数には交差エントロピー誤差
* 二値分類の出力層にはシグモイド関数、損失関数には交差エントロピー誤差

$$L=-(t\log{y}+(1-t)\log{(1-y)})$$

In [4]:
import numpy as np

In [1]:
class Embedding:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None
        
    def forward(self, idx):
        W, = self.params
        self.idx = idx
        out = W[idx]#特定の行を抜き出す
        return out 
    
#     def backward(self, dout):
#         dW, = self.grads
#         dW[...] = 0
#         dW[self.idx] = dout
#         return None

#重複問題が発生するため、代入ではなく加算を行う。
    def backward(self, dout):
        dW, = self.grads
        dW[...] = 0
        np.add.at(dW, self.idx, dout)#Numpyの方が早い
        
        return None

In [2]:
class EmbeddingDot:
    def __init__(self, W):
        self.embed = Embedding(W)
        self.params = self.embed.params
        self.grads = self.embed.grads
        self.cache = None
        
    def forward(self, h, idx):
        target_W = self.embed.forward(idx)
        out = np.sum(target_W * h, axis=1)
        self.cache = (h, target_W)
        return out
    
    def backward(self, dout):
        h, target_W = self.cache
        dout = dout.reshape(dout.shape[0], 1)
        dtarget_W = dout * h
        self.embed.backward(dtarget_W)
        dh = dout * target_W
        return dh

#### Negative Sampling

In [6]:
print(np.random.choice(10))
words = ["you", "say", "goodbye", "I", "hello", "."]
print(np.random.choice(words))
print(np.random.choice(words, size=5))
print(np.random.choice(words, size=5, replace=False))
p = [0.5, 0.1, 0.05, 0.2, 0.05, 0.1]
print(np.random.choice(words, p=p))

5
say
['say' 'say' 'say' 'you' 'you']
['.' 'you' 'say' 'hello' 'goodbye']
you
